In [62]:
import pandas as pd
import numpy as np
from sklearn.linear_model import (
    LinearRegression, LogisticRegression,
    LassoCV, LogisticRegressionCV
)
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from math import erf, sqrt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [26]:
DT = pd.read_csv("penn_jae.dat.txt", sep=r"\s+", header=0)        

print(DT)

        abdt  tg  inuidur1  inuidur2  female  black  hispanic  othrace  dep  \
0      10824   0        18        18       0      0         0        0    2   
1      10635   2         7         3       0      0         0        0    0   
2      10551   5        18         6       1      0         0        0    0   
3      10824   0         1         1       0      0         0        0    0   
4      10747   0        27        27       0      0         0        0    0   
...      ...  ..       ...       ...     ...    ...       ...      ...  ...   
13908  10831   5        27        27       0      0         0        0    0   
13909  10677   2         4         4       1      0         0        0    0   
13910  10817   4         4         4       0      0         0        0    0   
13911  10691   0        27        27       0      0         0        0    0   
13912  10677   5        25        25       0      0         0        0    0   

       q1  ...  q5  q6  recall  agelt35  agegt54  d

### Limpiar data

In [29]:
# 1. Normaliza nombres a minúsculas
DT = DT.rename(columns=str.lower)

# 2. Columnas requeridas
req_base = ["tg", "inuidur1", "dep"]
req_x_core = ["female", "black", "othrace", "q2", "q3", "q4", "q5", "q6",
              "recall", "durable", "nondurable", "lusd", "husd"]

missing = set(req_base + req_x_core) - set(DT.columns)
if missing:
    raise ValueError(f"Faltan columnas: {', '.join(missing)}")

# 3. Manejo de edad
has_agelt35 = "agelt35" in DT.columns
has_agegt54 = "agegt54" in DT.columns
has_age     = "age"     in DT.columns

if not (has_agelt35 and has_agegt54):
    if has_age:
        DT["agelt35"] = (DT["age"] < 35).astype(int)
        DT["agegt54"] = (DT["age"] > 54).astype(int)
    else:
        raise ValueError("No existen 'agelt35'/'agegt54' ni 'age' para derivarlas.")


# 4. Filtra tg == 0 o tg == 4
DT = DT[DT["tg"].isin([0, 4])].copy()

# 5. Tratamiento y outcome
DT["T4"] = (DT["tg"] == 4).astype(int)
DT["y"]  = np.log(DT["inuidur1"])

# 6. Dummies para dep (baseline dep_0)

DT["dep"] = DT["dep"].astype(int)
DT["dep_0"] = (DT["dep"] == 0).astype(int)
DT["dep_1"] = (DT["dep"] == 1).astype(int)
DT["dep_2"] = (DT["dep"] == 2).astype(int)

# 7. Define X como pide el enunciado

x_vars = [
    "female","black","othrace",
    "dep_1","dep_2",
    "q2","q3","q4","q5","q6",
    "recall","agelt35","agegt54",
    "durable","nondurable","lusd","husd"
]

missing_x = set(x_vars) - set(DT.columns)
if missing_x:
    raise ValueError(f"Faltan columnas de X: {', '.join(missing_x)}")

# 8. Dataset final
use_cols = ["y", "T4"] + x_vars
DT = DT[use_cols].dropna()

y = DT["y"].to_numpy()
d = DT["T4"].to_numpy()
X = DT[x_vars].to_numpy()
n = len(DT)

print(f"Muestra final: {n} filas, {X.shape[1]} predictores.")

Muestra final: 5099 filas, 17 predictores.


### Utilidades

In [33]:
def rmse(a, b):
    a = np.asarray(a)
    b = np.asarray(b)
    return np.sqrt(np.mean((a - b)**2))

def plm_theta_se(y_tilde, d_tilde):
    y_tilde = np.asarray(y_tilde)
    d_tilde = np.asarray(d_tilde)

    theta = np.sum(d_tilde * y_tilde) / np.sum(d_tilde * d_tilde)

    psi = (y_tilde - d_tilde * theta) * d_tilde

    se = np.sqrt(np.mean(psi**2) / (len(y_tilde) * (np.mean(d_tilde**2)**2)))

    return {
        "theta": theta,
        "se": se
    }

### Learners

In [ ]:
# OLS y LOGIT
def fit_y_ols(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    model = LinearRegression()
    model.fit(X, y)
    return model

def pred_y_ols(fit, X):
    X = np.asarray(X)
    return fit.predict(X)


def fit_d_logit(X, d):
    X = np.asarray(X)
    d = np.asarray(d)
    # equivalente a glm binomial (logit)
    model = LogisticRegression(solver="lbfgs", max_iter=1000)
    model.fit(X, d)
    return model

def pred_d_logit(fit, X):
    X = np.asarray(X)
    # probabilidad P(D=1 | X)
    return fit.predict_proba(X)[:, 1]


# LASSO (regresión y logit)
def fit_y_lasso(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    # cv.glmnet con family="gaussian", alpha=1
    model = LassoCV(cv=5, random_state=0)
    model.fit(X, y)
    return model
def pred_y_lasso(fit, X):
    X = np.asarray(X)
    return fit.predict(X)
def fit_d_lasso(X, d):
    X = np.asarray(X)
    d = np.asarray(d)
    # cv.glmnet con family="binomial", alpha=1
    model = LogisticRegressionCV(
        penalty="l1",
        solver="saga",
        cv=5,
        max_iter=10000
    )
    model.fit(X, d)
    return model
def pred_d_lasso(fit, X):
    X = np.asarray(X)
    # probabilidad P(D=1 | X)
    return fit.predict_proba(X)[:, 1]


# Random Forest
def fit_y_rf(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    model = RandomForestRegressor(
        n_estimators=1000,
        max_features="sqrt",
        min_samples_leaf=5,
        random_state=1
    )
    model.fit(X, y)
    return model
def pred_y_rf(fit, X):
    X = np.asarray(X)
    return fit.predict(X)
def fit_d_rf(X, d):
    X = np.asarray(X)
    d = np.asarray(d)
    model = RandomForestClassifier(
        n_estimators=1000,
        max_features="sqrt",
        min_samples_leaf=5,
        random_state=1
    )
    model.fit(X, d)
    return model
def pred_d_rf(fit, X):
    X = np.asarray(X)
    # probabilidad P(D=1 | X)
    proba = fit.predict_proba(X)
    return proba[:, 1]


# Neural net
def fit_y_nn(X, y, size=4, decay=1e-4, maxit=500):
    X = np.asarray(X)
    y = np.asarray(y)
    model = MLPRegressor(
        hidden_layer_sizes=(size,),
        alpha=decay,
        max_iter=maxit,
        activation="relu" 
    )
    model.fit(X, y)
    return model
def pred_y_nn(fit, X):
    X = np.asarray(X)
    return fit.predict(X)
def fit_d_nn(X, d, size=3, decay=1e-4, maxit=500):
    X = np.asarray(X)
    d = np.asarray(d)
    model = MLPClassifier(
        hidden_layer_sizes=(size,),
        alpha=decay,
        max_iter=maxit
    )
    model.fit(X, d)
    return model
def pred_d_nn(fit, X):
    X = np.asarray(X)
    proba = fit.predict_proba(X)
    if proba.shape[1] == 2:
        return proba[:, 1]
    return proba[:, -1]

### DML con Cross fitting

In [43]:
def dml_plm(  y,  d,  X, K=2,ml_y=None,   ml_d=None,  return_nuisance_rmse=True):
    if ml_y is None:
        ml_y = {"fit": fit_y_lasso, "pred": pred_y_lasso}
    if ml_d is None:
        ml_d = {"fit": fit_d_lasso, "pred": pred_d_lasso}

    y = np.asarray(y).ravel()
    d = np.asarray(d).ravel()
    X = np.asarray(X)

    n = len(y)

   
    fold_ids = np.tile(np.arange(K), int(np.ceil(n / K)))[:n]
    np.random.shuffle(fold_ids)
    folds = fold_ids

    m_hat = np.full(n, np.nan)
    g_hat = np.full(n, np.nan)
    rmse_y_folds = []
    rmse_d_folds = []

    for k in range(K):
        I_tr = folds != k
        I_te = folds == k

        fit_m = ml_y["fit"](X[I_tr, :], y[I_tr])
        fit_g = ml_d["fit"](X[I_tr, :], d[I_tr])

        m_hat[I_te] = ml_y["pred"](fit_m, X[I_te, :])
        g_hat[I_te] = ml_d["pred"](fit_g, X[I_te, :])

        if return_nuisance_rmse:
            rmse_y_folds.append(rmse(y[I_te], m_hat[I_te]))
            rmse_d_folds.append(rmse(d[I_te], g_hat[I_te]))

    y_tilde = y - m_hat
    d_tilde = d - g_hat

    est = plm_theta_se(y_tilde, d_tilde)

    out = {
        "theta": est["theta"],
        "se": est["se"]
    }

    if return_nuisance_rmse:
        out["rmse_y"] = float(np.mean(rmse_y_folds))
        out["rmse_d"] = float(np.mean(rmse_d_folds))

    return out

### DML sin crossfiting

In [47]:
def dml_plm_no_cf(y,d,X, K=2, ml_y=None, ml_d=None, return_nuisance_rmse=True):
 
    if ml_y is None:
        ml_y = {"fit": fit_y_lasso, "pred": pred_y_lasso}
    if ml_d is None:
        ml_d = {"fit": fit_d_lasso, "pred": pred_d_lasso}

    y = np.asarray(y).ravel()
    d = np.asarray(d).ravel()
    X = np.asarray(X)

    n = len(y)

    fold_ids = np.tile(np.arange(K), int(np.ceil(n / K)))[:n]
    np.random.shuffle(fold_ids)
    folds = fold_ids

    m_hat = np.full(n, np.nan)
    g_hat = np.full(n, np.nan)

    rmse_y_folds = []
    rmse_d_folds = []

    for k in range(K):
        I_k = (folds == k)

        fit_m = ml_y["fit"](X[I_k, :], y[I_k])
        fit_g = ml_d["fit"](X[I_k, :], d[I_k])

        m_hat[I_k] = ml_y["pred"](fit_m, X[I_k, :])
        g_hat[I_k] = ml_d["pred"](fit_g, X[I_k, :])

        if return_nuisance_rmse:
            rmse_y_folds.append(rmse(y[I_k], m_hat[I_k]))
            rmse_d_folds.append(rmse(d[I_k], g_hat[I_k]))

    y_tilde = y - m_hat
    d_tilde = d - g_hat

    est = plm_theta_se(y_tilde, d_tilde)

    out = {
        "theta": est["theta"],
        "se": est["se"]
    }

    if return_nuisance_rmse:
        out["rmse_y"] = float(np.mean(rmse_y_folds))
        out["rmse_d"] = float(np.mean(rmse_d_folds))

    return out

### Ejecutar CF y no CF con multiples modelos

In [52]:
# Normal CDF para el p-valor
def normal_cdf(z):
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

# Definición de learners (igual que en R)
learners = {
    "OLS+LOGIT": {
        "ml_y": {"fit": fit_y_ols,   "pred": pred_y_ols},
        "ml_d": {"fit": fit_d_logit, "pred": pred_d_logit},
    },
    "LASSO": {
        "ml_y": {"fit": fit_y_lasso, "pred": pred_y_lasso},
        "ml_d": {"fit": fit_d_lasso, "pred": pred_d_lasso},
    },
    "RF": {
        "ml_y": {"fit": fit_y_rf,    "pred": pred_y_rf},
        "ml_d": {"fit": fit_d_rf,    "pred": pred_d_rf},
    },
    "NN": {
        "ml_y": {"fit": fit_y_nn,    "pred": pred_y_nn},
        "ml_d": {"fit": fit_d_nn,    "pred": pred_d_nn},
    },
}


def run_block(fun, y, d, X, K, learners):
    rows = []

    for name, ml in learners.items():
        # set.seed(42) en cada iteración, como en R
        np.random.seed(42)

        est = fun(
            y=y,
            d=d,
            X=X,
            K=K,
            ml_y=ml["ml_y"],
            ml_d=ml["ml_d"],
            return_nuisance_rmse=True
        )

        theta = est["theta"]
        se    = est["se"]
        z     = theta / se
        pval  = 2 * (1 - normal_cdf(abs(z)))

        rows.append({
            "Method": name,
            "theta": theta,
            "se": se,
            "pval": pval,
            "rmse_y": est["rmse_y"],
            "rmse_d": est["rmse_d"],
        })

    return pd.DataFrame(rows)


# Ejecutar con y, d, X ya definidos
K = 2

tab_cf = run_block(dml_plm, y, d, X, K, learners)
tab_cf["CrossFitting"] = "Yes"

tab_nocf = run_block(dml_plm_no_cf, y, d, X, K, learners)
tab_nocf["CrossFitting"] = "No"

results_all = pd.concat([tab_cf, tab_nocf], ignore_index=True)
results_all = results_all.sort_values(["CrossFitting", "Method"])

print(results_all)

      Method     theta        se      pval    rmse_y    rmse_d CrossFitting
5      LASSO -0.072562  0.035092  0.038661  1.188601  0.473551           No
7         NN -0.070317  0.035075  0.044986  1.189861  0.473895           No
4  OLS+LOGIT -0.071796  0.035077  0.040676  1.188244  0.472959           No
6         RF -0.071405  0.035112  0.041988  1.136144  0.453549           No
1      LASSO -0.077234  0.035205  0.028248  1.197006  0.474995          Yes
3         NN -0.075268  0.035470  0.033835  1.206131  0.475127          Yes
0  OLS+LOGIT -0.075519  0.035218  0.032005  1.197143  0.474581          Yes
2         RF -0.074126  0.035016  0.034269  1.198461  0.477559          Yes


### OLS con controles como benchmark

In [64]:
colnames = ["y", "d"] + [f"x{i}" for i in range(X.shape[1])]
df_full = pd.DataFrame(np.column_stack([y, d, X]), columns=colnames)
formula = "y ~ d + " + " + ".join(df_full.columns[2:])
ols_full = smf.ols(formula, data=df_full).fit()

theta_ols_controls = ols_full.params["d"]
se_ols_controls    = ols_full.bse["d"]
pval_ols_controls  = ols_full.pvalues["d"]

rmse_y = rmse(df_full["y"], ols_full.predict(df_full))

logit_full = smf.glm("d ~ " + " + ".join(df_full.columns[2:]),
                     data=df_full,
                     family=sm.families.Binomial()).fit()

pred_d = logit_full.predict(df_full)
rmse_d = rmse(df_full["d"], pred_d)

ols_row = pd.DataFrame([{
    "CrossFitting": "N/A",
    "Method": "OLS with controls",
    "theta": theta_ols_controls,
    "se": se_ols_controls,
    "pval": pval_ols_controls,
    "rmse_y": rmse_y,
    "rmse_d": rmse_d
}])

results_all = pd.concat([results_all, ols_row], ignore_index=True)
results_all = results_all.sort_values(["CrossFitting", "Method"])

print(results_all)

              Method     theta        se      pval    rmse_y    rmse_d  \
8  OLS with controls -0.072576  0.035270  0.039671  1.189979  0.473362   
0              LASSO -0.072562  0.035092  0.038661  1.188601  0.473551   
1                 NN -0.070317  0.035075  0.044986  1.189861  0.473895   
2          OLS+LOGIT -0.071796  0.035077  0.040676  1.188244  0.472959   
3                 RF -0.071405  0.035112  0.041988  1.136144  0.453549   
4              LASSO -0.077234  0.035205  0.028248  1.197006  0.474995   
5                 NN -0.075268  0.035470  0.033835  1.206131  0.475127   
6          OLS+LOGIT -0.075519  0.035218  0.032005  1.197143  0.474581   
7                 RF -0.074126  0.035016  0.034269  1.198461  0.477559   

  CrossFitting  
8          N/A  
0           No  
1           No  
2           No  
3           No  
4          Yes  
5          Yes  
6          Yes  
7          Yes  


### Seleccion de modelo (CF) y estimacion final

In [66]:
tab_cf_sorted = tab_cf.sort_values("se")
best_cf = tab_cf_sorted.iloc[0]
print(best_cf)

def run_final(method=None):
    if method is None:
        method = best_cf["Method"]

    ml = learners[method]

    out = dml_plm(
        y=y,
        d=d,
        X=X,
        K=K,
        ml_y=ml["ml_y"],
        ml_d=ml["ml_d"],
        return_nuisance_rmse=True
    )

    z = out["theta"] / out["se"]
    pval = 2 * (1 - normal_cdf(abs(z)))

    print(f"\nFinal DML (CF) con {method}")
    print(f"theta={out['theta']:.4f}, se={out['se']:.4f}, pval={pval:.4g}")

    out["pval"] = pval
    return out

Method                RF
theta          -0.074126
se              0.035016
pval            0.034269
rmse_y          1.198461
rmse_d          0.477559
CrossFitting         Yes
Name: 2, dtype: object


### Resumen

In [72]:
def print_table(TAB, title="Resultados"):
    print(f"\n{title}")
    print("-------------------------------------------------------------")

    df_print = TAB.copy()

    df_print["theta"]  = df_print["theta"].round(4)
    df_print["se"]     = df_print["se"].round(4)
    df_print["pval"]   = df_print["pval"].apply(lambda x: float(f"{x:.3g}"))
    df_print["rmse_y"] = df_print["rmse_y"].round(4)
    df_print["rmse_d"] = df_print["rmse_d"].round(4)

    cols = ["CrossFitting", "Method", "theta", "se", "pval", "rmse_y", "rmse_d"]
    print(df_print[cols])

print_table(tab_cf,   "Table A. DML con cross-fitting")
print_table(tab_nocf, "Table B. DML sin cross-fitting")
print_table(results_all, "Appendix. Todos los modelos (incluye OLS con controles)")


Table A. DML con cross-fitting
-------------------------------------------------------------
  CrossFitting     Method   theta      se    pval  rmse_y  rmse_d
0          Yes  OLS+LOGIT -0.0755  0.0352  0.0320  1.1971  0.4746
1          Yes      LASSO -0.0772  0.0352  0.0282  1.1970  0.4750
2          Yes         RF -0.0741  0.0350  0.0343  1.1985  0.4776
3          Yes         NN -0.0753  0.0355  0.0338  1.2061  0.4751

Table B. DML sin cross-fitting
-------------------------------------------------------------
  CrossFitting     Method   theta      se    pval  rmse_y  rmse_d
0           No  OLS+LOGIT -0.0718  0.0351  0.0407  1.1882  0.4730
1           No      LASSO -0.0726  0.0351  0.0387  1.1886  0.4736
2           No         RF -0.0714  0.0351  0.0420  1.1361  0.4535
3           No         NN -0.0703  0.0351  0.0450  1.1899  0.4739

Appendix. Todos los modelos (incluye OLS con controles)
-------------------------------------------------------------
  CrossFitting             Method

# 10) Respuestas en Markdown
# Answers

## PLM and DML
We estimate the partially linear model
$ y = \\theta d + g_0(X) + \\varepsilon, \\quad d = m_0(X) + \\nu.$
DML uses cross-fitting to build out-of-sample residuals
$\\tilde y = y - \\hat g(X),\\; \\tilde d = d - \\hat m(X)$
and
$\\hat\\theta = \\frac{\\sum_i \\tilde d_i\\tilde y_i}{\\sum_i \\tilde d_i^2}$,
with IF-based standard errors.

## Cross-fitting vs no cross-fitting
- RMSE for predicting $y$ and $d$ is usually **smaller** without cross-fitting due to in-sample optimism.
- Lower RMSE there does **not** mean better causal inference; it reflects **overfitting** of nuisances.
- Sin cross-fitting, el sesgo de regularización se filtra al estimando y genera **sesgo** y **inferencias no conservadoras**.

## Selected model
Choose the CF method with the smallest SE in Table A and report its $\\hat\\theta$ as the final effect.
")
### RF -0.0741  0.0350  0.0343  1.1985  0.4776
